In [1]:
#pip3 install google.colab
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

###  Libraries

In [2]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors

import torch

import os
import sys

from MLP import MLP
from Datasets.Datasets_Functions import *
from Visualization import Visualization
from Evaluations.Evaluation import Evaluator

### Global Variables

In [3]:
path_to_module = './' # 'C:\\Users\\PAPASOFT INC\\Desktop\\SOFT_SIL'
sys.path.append(path_to_module)
os.environ['OMP_NUM_THREADS'] = '6'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

### Visualization

In [4]:
visualization = Visualization()

### Load datasets

In [5]:
hashmap_path = path_to_module + 'Datasets/'
hashmap = get_hashmap(hashmap_path)    
print(hashmap.keys())

dataset_name = 'emnist_balanced_digits'
dataset_properties = hashmap[dataset_name]
print(dataset_properties)

batch_size = dataset_properties['batch_size'] #= 30
n_clusters = dataset_properties['n_clusters'] = 10
dataloader, input_dim, data_np, labels = function_get_dataset(dataset_name, dataset_properties)

print('Data_Shape is:', input_dim)
print('Batches Number is:', len(dataloader))

dict_keys(['tcga', 'emnist_balanced_letters', 'emnist_mnist', 'emnist_balanced_digits', 'dermatology', 'ecoil', 'iris', 'rings', 'wine', 'australian', 'moons', 'squeezed_gauss', 'gauss_densities', 'pendigits', 'fashionmnist', '3dspheres', '20_newsgroups', 'coil20', 'cifar10', 'stl10', 'r15', 'r3', 'olivetti_faces', 'r100', '10x73k', 'usps', 'kmnist', 'reuters_4'])
{'batch_size': 1024, 'n_clusters': 10, 'module_name': 'Datasets.Datasets'}


[nltk_data] Downloading package punkt to C:\Users\PAPASOFT
[nltk_data]     INC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\PAPASOFT
[nltk_data]     INC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data_Shape is: 1
Batches Number is: 28


### Parameters

In [6]:
# Batch Size and Number of Clusters
batch_size = dataset_properties['batch_size']
n_clusters = dataset_properties['n_clusters']

# Training Epochs and Learning Rate
lr = 1e-3
n_epochs = 50
entr_lambda = 0.02

kmeans_initialization = True

### Create MLP model

In [7]:
mlp = MLP(device=device, n_clusters=n_clusters, input_dim=input_dim)
mlp.set_training_variables(dataloader=dataloader, batch_size=batch_size, n_epochs=n_epochs, lr=lr, entr_lambda=entr_lambda)
mlp.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
mlp.set_path()

mlp = mlp.to(device)

### Initialize the clustering layer using k-means

In [8]:
n_inits = 10
if kmeans_initialization: mlp.kmeans_initialization(n_inits)

ValueError: Found array with dim 4. KMeans expected <= 2.

### Training the MLP

In [ ]:
data, labels, clustering = mlp.train()
cluster_centers = mlp.get_clustering_layer_centers().cpu().detach().numpy()
data_dir_path = mlp.data_dir_path

### $k$-means evaluation

In [ ]:
kmeans = KMeans(n_clusters=n_clusters, n_init=10).fit(data)
evaluator = Evaluator()
evaluator.evaluate_model(data, labels, kmeans.labels_)

In [ ]:
visualization.plot(data, labels, clustering, cluster_centers, mlp.data_dir_path)

In [ ]:
#visualization.makeExcel()
visualization.plot_tsne(data, labels, clustering, cluster_centers, mlp.data_dir_path)